## 🎅Santa 2025 Optimization and Pattern Visualization Notebook🎅
- Load the best `submission.csv` and (optionally) upgrade examples 2-5 with the heavy-duty optimizer
- Rebuild the geometric tree model in pure Python so every visualization is reproducible, precise, and comparable to the starter notebook
- Craft neon-style Plotly scenes plus density fields to uncover layout patterns for the examples
- We want to understand which patterns appear, so we don't have to waste CPU resources on big examples

### ✨You need to fork and run the notebook to use the visualizer interactively✨
#### The next images are static examples of the visualizer. Run the code and scroll to the bottom of this notebook to use the interactive visualizer.

In [ ]:
from IPython.display import Image, display

#display(Image('/kaggle/input/santa2025visualizerimages/configuration_5.png'))
display(Image('/kaggle/input/santa2025visualizerimages/first_10_gallery.png'))
display(Image('/kaggle/input/santa2025visualizerimages/interactive_visualizer.png'))

## Challenge Context
Santa ships between 1 and 200 toy trees per parcel. For each configuration we must report the (x, y, deg) pose for every tree, and the leaderboard adds up `(side^2 / n)` where *side* is the edge of the minimum bounding square. Smaller squares win. Any overlap fails validation, and all coordinates must stay within [-100, 100].

In [ ]:
# REPLACE WITH YOUR SUBMISSION
submission_path = '/kaggle/input/santa2025submission/submission.csv'

In [ ]:

from pathlib import Path
from decimal import getcontext

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from IPython.display import display

pd.set_option('display.float_format', '{:.9f}'.format)
px.defaults.template = 'plotly'
px.defaults.color_continuous_scale = 'Viridis'
px.defaults.color_discrete_sequence = px.colors.qualitative.Safe

In [ ]:
submission_raw = pd.read_csv(submission_path)

def as_float(series: pd.Series) -> pd.Series:
    return series.astype(str).str.replace('^s', '', regex=True).astype(float)

submission = submission_raw.copy()
for col in ['x', 'y', 'deg']:
    submission[col] = as_float(submission_raw[col])

submission['n'] = submission['id'].str.slice(0, 3).astype(int)
submission['tree_idx'] = submission['id'].str.split('_').str[1].astype(int)

print(f'Loaded {len(submission):,} tree placements spanning {submission.nunique()} configurations.')
display(submission.head())

## Optional: optimize examples with low number of trees
`parallel_optimizer.py` contains a CPU-heavy search routine (simulated annealing + grid seeding). I run it locally on my machine for a few hours on examples with lower number of trees. Flip the flag if you want improved placements that still feed into the visual analysis. This code was used to find best solutions for smaller exampels.

In [ ]:
RUN_OPTIMIZATION = False  # Set to True on Kaggle if you want fresh solutions for ids [2, 3, 4, 5]
OPTIMIZATION_IDS = [2, 3, 4, 5]

import sys
sys.path.append('/kaggle/input/santa2025paralleloptimizer')

if RUN_OPTIMIZATION:
    import multiprocessing as mp
    from parallel_optimizer import parallel_optimize, create_submission

    existing_df = submission_raw.copy()
    worker_count = mp.cpu_count()
    print(f'Launching optimizer on ids {OPTIMIZATION_IDS} with {worker_count} workers per example...')
    optimized_results, sources = parallel_optimize(
        config_ids=OPTIMIZATION_IDS,
        attempt_workers=worker_count,
        existing_df=existing_df,
    )
    optimized_file = 'submission.csv'
    create_submission(
        optimized_results,
        optimized_ids=OPTIMIZATION_IDS,
        input_csv=str(submission_path),
        output_csv=optimized_file,
    )
    submission_path = Path(optimized_file)
    submission_raw = pd.read_csv(submission_path)
    for col in ['x', 'y', 'deg']:
        submission[col] = as_float(submission_raw[col])
    submission['n'] = submission['id'].str.slice(0, 3).astype(int)
    submission['tree_idx'] = submission['id'].str.split('_').str[1].astype(int)
    print(f'Re-loaded optimized submission from {optimized_file}.')
else:
    submission_raw = pd.read_csv(submission_path)
    submission_raw.to_csv('submission.csv')
    print('Optimization skipped. Flip RUN_OPTIMIZATION to True to explore improved packs.')

## Rebuilding the geometric model
To reproduce the starter visualization (and support fancy Plotly art) we recreate the exact tree polygon: three tiers plus a trunk with the same dimensions (0.7/0.4/0.25 widths and 0.8 height). Every polygon is rotated around the trunk center, then translated to `(x, y)` from the submission.

In [ ]:
getcontext().prec = 28

TREE_TEMPLATE = [
    (0.0, 0.8),
    (0.125, 0.5),
    (0.0625, 0.5),
    (0.2, 0.25),
    (0.1, 0.25),
    (0.35, 0.0),
    (0.075, 0.0),
    (0.075, -0.2),
    (-0.075, -0.2),
    (-0.075, 0.0),
    (-0.35, 0.0),
    (-0.1, 0.25),
    (-0.2, 0.25),
    (-0.0625, 0.5),
    (-0.125, 0.5),
]

def build_tree_polygon(cx: float, cy: float, angle_deg: float) -> Polygon:
    base = Polygon(TREE_TEMPLATE)
    rotated = affinity.rotate(base, angle_deg, origin=(0, 0), use_radians=False)
    return affinity.translate(rotated, xoff=cx, yoff=cy)

def compute_bounding_square(polygons):
    if not polygons:
        return 0.0, (0.0, 0.0, 0.0, 0.0)
    union = unary_union(polygons)
    minx, miny, maxx, maxy = union.bounds
    width = maxx - minx
    height = maxy - miny
    side = max(width, height)
    if width >= height:
        square_x = minx
        square_y = miny - (side - height) / 2
    else:
        square_x = minx - (side - width) / 2
        square_y = miny
    return side, (square_x, square_y, square_x + side, square_y + side)

configuration_cache = {}

def get_configuration(n: int):
    if n not in configuration_cache:
        group = submission[submission['n'] == n]
        polygons = [
            build_tree_polygon(row.x, row.y, row.deg)
            for row in group.itertuples()
        ]
        side, square = compute_bounding_square(polygons)
        configuration_cache[n] = {
            'polygons': polygons,
            'side': side,
            'square': square,
            'tree_count': len(polygons),
        }
    return configuration_cache[n]

In [ ]:
stats_rows = []
for n in sorted(submission['n'].unique()):
    cfg = get_configuration(n)
    score = (cfg['side'] ** 2) / n
    stats_rows.append({
        'n': n,
        'trees': cfg['tree_count'],
        'side': cfg['side'],
        'score': score,
    })

stats_df = pd.DataFrame(stats_rows)
display(stats_df.head())
print(f'Best normalized score so far: {stats_df["score"].min():.6f}')

### Which configurations are best?
Low `(side^2 / n)` means every centimeter is used. The table below lists the most compact and the loosest packings currently in `submission.csv`.

In [ ]:
print('Top 5 tightest packings:')
display(stats_df.nsmallest(5, 'score'))
print('Bottom 5 loosest packings:')
display(stats_df.nlargest(5, 'score'))

## Neon configuration explorer

This notebook acts as an example visualizer, walking through how neon overlays and bounding squares expose layout quality for each configuration.

In [ ]:
def neon_rgba(rgb: str, alpha: float = 0.4) -> str:
    if rgb.startswith('rgb'):
        comps = rgb[rgb.find('(') + 1: rgb.find(')')].split(',')
        r, g, b = [int(float(c)) for c in comps]
        return f'rgba({r},{g},{b},{alpha})'
    return rgb

def make_configuration_figure(n: int) -> go.Figure:
    cfg = get_configuration(n)
    polygons = cfg['polygons']
    colors = px.colors.sample_colorscale('Plasma', max(len(polygons), 2))
    traces = []
    for idx, (poly, color) in enumerate(zip(polygons, colors)):
        x, y = map(list, poly.exterior.xy)
        traces.append(go.Scatter(
            x=x,
            y=y,
            mode='lines',
            fill='toself',
            line=dict(color=color, width=2),
            fillcolor=neon_rgba(color, 0.45),
            hovertemplate=f'Tree {idx} \u2192 (x=%{{x:.3f}}, y=%{{y:.3f}})<extra></extra>',
            showlegend=False,
        ))
    centers = submission.loc[submission['n'] == n, ['x', 'y']].to_numpy()
    traces.append(go.Scatter(
        x=centers[:, 0],
        y=centers[:, 1],
        mode='markers',
        marker=dict(color='#10b981', size=7, opacity=0.85, line=dict(color='#fde68a', width=1)),
        hovertemplate='Center (x=%{x:.3f}, y=%{y:.3f})<extra></extra>',
        showlegend=False,
    ))
    square = cfg['square']
    square_x = [square[0], square[2], square[2], square[0], square[0]]
    square_y = [square[1], square[1], square[3], square[3], square[1]]
    traces.append(go.Scatter(
        x=square_x,
        y=square_y,
        mode='lines',
        line=dict(color='#fb7185', width=3, dash='dash'),
        hovertemplate='Bounding square edge<extra></extra>',
        showlegend=False,
    ))
    fig = go.Figure(traces)
    fig.update_layout(
        title=f'n={n} (side={cfg["side"]:.4f})',
        paper_bgcolor='#fff7ed',
        plot_bgcolor='#fff7ed',
        font=dict(color='#1d1b16'),
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False, scaleanchor='x', scaleratio=1),
        margin=dict(l=20, r=20, t=60, b=20),
    )
    fig.add_annotation(
        text=f'Score = {(cfg["side"] ** 2 / n):.6f}',
        xref='paper', yref='paper', x=0.01, y=1.08, showarrow=False,
        font=dict(color='#b45309', size=12)
    )
    score = (cfg['side'] ** 2) / n
    print(f'n={n}: side={cfg["side"]:.6f}, score={score:.6f}')
    return fig

# Display static example for n=5
fig = make_configuration_figure(5)
display(fig)

## First 10 configurations at a glance
The gallery below arranges the smallest nn layouts (1–10) into a single canvas so you can compare shapes, rotations, and bounding squares without moving the slider.

In [ ]:
def render_first_ten_examples():
    first_ten = sorted(submission['n'].unique())[:10]
    fig = make_subplots(
        rows=2,
        cols=5,
        subplot_titles=[f'n = {n}' for n in first_ten],
        horizontal_spacing=0.03,
        vertical_spacing=0.08,
    )
    for idx, n in enumerate(first_ten):
        cfg = get_configuration(n)
        row = idx // 5 + 1
        col = idx % 5 + 1
        colors = px.colors.sample_colorscale('Viridis', max(len(cfg['polygons']), 2))
        for poly, color in zip(cfg['polygons'], colors):
            x, y = map(list, poly.exterior.xy)
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=y,
                    mode='lines',
                    fill='toself',
                    line=dict(color=color, width=1.3),
                    fillcolor=neon_rgba(color, 0.35),
                    hoverinfo='skip',
                    showlegend=False,
                ),
                row=row,
                col=col,
            )
        square = cfg['square']
        square_x = [square[0], square[2], square[2], square[0], square[0]]
        square_y = [square[1], square[1], square[3], square[3], square[1]]
        fig.add_trace(
            go.Scatter(
                x=square_x,
                y=square_y,
                mode='lines',
                line=dict(color='#fb7185', width=2, dash='dash'),
                hoverinfo='skip',
                showlegend=False,
            ),
            row=row,
            col=col,
        )
        centers = submission.loc[submission['n'] == n, ['x', 'y']].to_numpy()
        fig.add_trace(
            go.Scatter(
                x=centers[:, 0],
                y=centers[:, 1],
                mode='markers',
                marker=dict(color='#10b981', size=4, opacity=0.8),
                hoverinfo='skip',
                showlegend=False,
            ),
            row=row,
            col=col,
        )
        score = (cfg['side'] ** 2) / n
        print(f'Gallery chart n={n}: side={cfg["side"]:.6f}, score={score:.6f}')
        fig.add_annotation(
            text=f"side={cfg['side']:.3f}<br>score={score:.4f}",
            xref=f'x{idx + 1}',
            yref=f'y{idx + 1}',
            x=square[0] + (square[2] - square[0]) * 0.05,
            y=square[3] - (square[3] - square[1]) * 0.05,
            showarrow=False,
            font=dict(color='#92400e', size=10),
            bgcolor='rgba(255,255,255,0.75)',
            borderpad=2,
        )
        fig.update_xaxes(showgrid=False, zeroline=False, row=row, col=col)
        fig.update_yaxes(showgrid=False, zeroline=False, scaleratio=1, scaleanchor=f'x{idx + 1}', row=row, col=col)
    fig.update_layout(
        title='First 10 configurations overview',
        height=850,
        width=1500,
        paper_bgcolor='#fffaf0',
        plot_bgcolor='#fffaf0',
        font=dict(color='#1f2937'),
        margin=dict(l=30, r=30, t=120, b=40),
    )
    return fig

# Render static gallery of first 10 examples
fig = render_first_ten_examples()
display(fig)

Below we render every tree polygon with glowing fills, overlay the bounding square, and annotate the score. Use the interactive interface to browse through the examples.

In [ ]:
def interactive_view(n: int = 5):
    make_configuration_figure(int(n)).show()

try:
    import ipywidgets as widgets

    max_n = min(200, int(submission['n'].max()))
    slider = widgets.IntSlider(
        min=1,
        max=max_n,
        step=1,
        value=5,
        description='n',
        continuous_update=False,
        layout=widgets.Layout(width='320px')
    )
    direct_input = widgets.BoundedIntText(
        min=1,
        max=max_n,
        value=slider.value,
        description='Jump',
        layout=widgets.Layout(width='200px')
    )
    btn_prev = widgets.Button(description='◀', layout=widgets.Layout(width='90px'))
    btn_next = widgets.Button(description='▶', layout=widgets.Layout(width='90px'))
    btn_prev.style.button_color = '#fb7185'
    btn_next.style.button_color = '#34d399'
    for btn in (btn_prev, btn_next):
        btn.style.font_weight = 'bold'
        btn.style.font_color = '#1d1b16'
    out = widgets.Output()

    def render(change=None):
        with out:
            out.clear_output(wait=True)
            interactive_view(int(slider.value))
        if direct_input.value != slider.value:
            direct_input.value = int(slider.value)

    def sync_text(change):
        if change['name'] == 'value' and change['new'] != slider.value:
            slider.value = int(change['new'])

    def shift(delta):
        slider.value = max(slider.min, min(slider.max, slider.value + delta))

    slider.observe(render, names='value')
    direct_input.observe(sync_text, names='value')
    btn_prev.on_click(lambda _: shift(-1))
    btn_next.on_click(lambda _: shift(1))

    controls = widgets.HBox(
        [btn_prev, slider, btn_next, direct_input],
        layout=widgets.Layout(align_items='center', justify_content='space-between')
    )
    display(widgets.VBox([controls, out]))
    render()
except Exception as exc:
    print(f'Interactive widgets unavailable ({exc}); static visualization shown above.')